<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/Software_anomaly_detection_%E7%9B%B8%E9%96%A2%E4%BF%82%E6%95%B026_xgboost_param_tuning_reloaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')

!pip install optuna #optunaのインストール

import optuna.integration.lightgbm as lgb_tune #LightGBMのパラメータチューニング
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)
# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error



# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [2]:
# data_train csvファイルの読み込み
train_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/train.zip (Unzipped Files)/train.csv',usecols=['Anomaly',
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [3]:
# data_test csvファイルの読み込み
test_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/test.zip (Unzipped Files)/test.csv',usecols=[
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [4]:
#スコア0.9279の予測データを読み込み
test_y = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded2.csv')

In [5]:
#Anomalyのみを取り出し値を四捨五入
test_y=test_y["Anomaly"]
test_y=test_y.round(0).astype(int)

In [6]:
#異常フラグのみのリストを作成
train_y=train_df["Anomaly"]

In [7]:
#異常フラグ、カテゴリを除いたリストを作成
train_x=train_df.drop(["Anomaly","host"],axis=1)
test_x=test_df.drop(["host"],axis=1)

In [8]:
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)

In [9]:
train_y = pd.Series(train_y)
test_y = pd.Series(test_y)

In [10]:
# ラベルエンコーディング（OrdinalEncoder）
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder


oe = preprocessing.OrdinalEncoder()
train_encoded = oe.fit_transform(train_df[['host']].values)
train_decoded = oe.inverse_transform(train_encoded)
test_encoded = oe.fit_transform(test_df[['host']].values)
test_decoded = oe.inverse_transform(test_encoded)

print('エンコード結果: ', [train_encoded,test_encoded])
print('元に戻す: ', [train_decoded,test_decoded])

エンコード結果:  [array([[0.],
       [0.],
       [0.],
       ...,
       [9.],
       [9.],
       [9.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [9.],
       [9.],
       [9.]])]
元に戻す:  [array([['lphost06'],
       ['lphost06'],
       ['lphost06'],
       ...,
       ['lphost18'],
       ['lphost18'],
       ['lphost18']], dtype=object), array([['lphost06'],
       ['lphost06'],
       ['lphost06'],
       ...,
       ['lphost18'],
       ['lphost18'],
       ['lphost18']], dtype=object)]


In [11]:
train_encoded_df = pd.DataFrame(train_encoded)
train_encoded_df.columns=['host']
test_encoded_df = pd.DataFrame(test_encoded)
test_encoded_df.columns=['host']

In [12]:
#エンコードしたカテゴリデータとカテゴリを除いた訓練データを結合
train_x = pd.concat([train_x, train_encoded_df],axis=1)
test_x = pd.concat([test_x, test_encoded_df],axis=1)

In [13]:
#trainデータとtestデータを結合
train2_x = pd.concat([train_x, test_x],axis=0)
train2_y = pd.concat([train_y, test_y],axis=0)

In [14]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train2_x))[0]
x_train, x_valid = train2_x.iloc[tr_idx], train2_x.iloc[va_idx]
y_train, y_valid = train2_y.iloc[tr_idx], train2_y.iloc[va_idx]


In [15]:
print(y_train)

0          0
2          0
3          0
4          0
5          0
          ..
2927226    0
2927228    0
2927229    0
2927230    0
2927231    0
Name: Anomaly, Length: 3754218, dtype: int64


In [16]:
print(x_train.shape)
print(x_valid.shape)
print(test_x.shape)

(3754218, 26)
(938555, 26)
(2927232, 26)


In [17]:
# 学習に使用するデータを設定
#lgb_train = lgb.Dataset(x_train, y_train)
#lgb_eval = lgb.Dataset(x_valid, y_valid,reference=lgb_train)

In [18]:
# xgboostの実装
# -----------------------------------
import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid,label=y_valid)
dtest = xgb.DMatrix(test_x)

In [33]:
# 学習データとバリデーションデータのスコアのモニタリング
# -----------------------------------
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma': 0.00014307741815837628,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 1.4034295969588149,
    'max_depth': 9,
    'subsample': 0.95,
    'colsample_bytree': 0.95,
    'random_state': 71,
    #上記で取得したパラメータ
}
num_round = 10000
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=100)

[0]	train-error:0.000977	eval-error:0.001031
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 100 rounds.
[1]	train-error:0.000667	eval-error:0.000726
[2]	train-error:0.000658	eval-error:0.000689
[3]	train-error:0.000637	eval-error:0.000676
[4]	train-error:0.000653	eval-error:0.000678
[5]	train-error:0.000608	eval-error:0.000633
[6]	train-error:0.000555	eval-error:0.000576
[7]	train-error:0.000542	eval-error:0.000564
[8]	train-error:0.000563	eval-error:0.000573
[9]	train-error:0.000533	eval-error:0.000553
[10]	train-error:0.000522	eval-error:0.000541
[11]	train-error:0.000471	eval-error:0.00049
[12]	train-error:0.000446	eval-error:0.000453
[13]	train-error:0.000409	eval-error:0.000419
[14]	train-error:0.000395	eval-error:0.000407
[15]	train-error:0.000346	eval-error:0.000358
[16]	train-error:0.000334	eval-error:0.000349
[17]	train-error:0.000309	eval-error:0.000322
[18]	train-error:0.000307	eval-error:

In [45]:
# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(y_valid, va_pred)
print(f'logloss: {score:.12f}')

logloss: nan


In [46]:
# 構築したモデルの保存
import pickle
file = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/xgboost_trained_model_26_param_tuning_test_reloaded5.pkl'
pickle.dump(model, open(file, 'wb'))

In [47]:
#from モジュール名 import クラス名（もしくは関数名や変数名）
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#matplotlibをinlineで表示するためのおまじない (plt.show()しなくていい)
%matplotlib inline

In [48]:
# 構築したモデルの読み込み
import pickle
filename = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/xgboost_trained_model_26_param_tuning_test_reloaded5.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [49]:
# submission csvファイルの読み込み
submission_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/data.zip (Unzipped Files)/sample_submission.csv')

In [50]:
id_df=submission_df.iloc[:,0]
id_df.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [51]:
pred=loaded_model.predict(dtest,ntree_limit=loaded_model.best_ntree_limit)

In [52]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,1.790803e-08
1,1.869294e-08
2,2.049332e-08
3,1.622603e-08
4,2.121737e-08


In [53]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','Anomaly']
output_df.head()

,id,Anomaly
0,0,1.790803e-08
1,1,1.869294e-08
2,2,2.049332e-08
3,3,1.622603e-08
4,4,2.121737e-08


In [54]:
output_df.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded5.csv',index=False)